# C.2. Multiperiod Trees and American Options

## Module: Options

### DRW New-Hire Learning Program

#### Summer 2022

Questions?
* Contact: Mark Hendricks
* hendricks@uchicago.edu

$$\newcommand{\Ern}{E^*}$$
$$\newcommand{\hedge}{\beta}$$
$$\newcommand{\dt}{\Delta t}$$

# A Multiperiod Tree

### Reminder on the One-period Tree

We have focused on a one-period (of uncertainty) tree for
* equities
* rates

We saw that the same computational approach applied to both, and it priced the option via
* no-arbitrage replication
* no-arbitrage risk-adjusted (risk-neutral) discounted expected values.


## Multiperiod Equity Tree

Continue the logic of the equity tree, and extend it to a second period.
* Continue to use $u$ and $d$ to denote multiplicative "up" and "down" factors.
* Fill out the underlying tree with $u$ and $d$ factors to match the volatility of the underlying.

See the figure below.

<img src="../refs/options/tree_multiperiod/two_period_form.png" width="700" />
Reference: John Hull, Chapter 13.

## Solution Method

#### Boundary (Terminal) Conditions

The contract determines the payoff at expiration as a known function of the underlying.
* This gives us the option value at the final nodes.

#### Moving Backward
From there, go backward in time one node.
* We have two separate, one-period trees.

The solution procedure works exactly as discussed before--same formulas, with the obvious adaptations.
* $f_{uu}$ and $f_{ud}$ instead of $f_u$ and $f_d$.
* $S_0uu$ and $S_0ud$ instead of $S_0u$ and $S_0d$.

# The Multiperiod Tree

#### Iterate
* Continue at each state of $T-i\dt$.
* Move back a time step, $\dt$, and continue.

#### Choosing a time-grid
The time-grid must be chosen fine enough to get convergence.
* Common to see $\dt$ chosen such that there are around 30 steps for a vanilla American option.
* Exotic options (barrier, knock-in, etc) may need many more steps.

If the time grid is chosen too coarsely, the solution will not converge to a reasonable solution.
* Furthermore, there may be issues of negative risk-neutral probabilities.
* If the variance in any given step is too large, the probabilities become extreme to try to match it.
* Small time-steps keep this from happening.

#### Specific underlying values
In the tree, we do not get to set the exact price values.
* We have been setting $u$ and $d$ to match volatility.
* For some options, specific underlying values are of importance.

For this flexibility, we would need to take a different approach which will lead to **trinomial trees.**

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '../cmds')
from options import *

### Inputs

In [2]:
## Market variables
r = .05
sigma = .25
So = 50

## Contract
T = 2
K = 55
funPayoff = lambda x: np.maximum(K-x,0)

## Solving variable
Nt = 2
uset = 1.2
dset = .8

dt = T/Nt

### Underlying Tree

In [3]:
tree, treeinfo = treeUnder(So,T,Nt,sigma=sigma)

tree.style.format('{:.2f}')

,0,1,2
0,50.00,64.20,82.44
1,nan,38.94,50.00
2,nan,nan,30.33


### Tree Rates

In [4]:
rates = r * np.ones(Nt)

Z = np.exp(-rates*dt)
A = np.exp(rates * dt)

pstar = (A - treeinfo.d)/(treeinfo.u-treeinfo.d)

print(f'Risk-neutral probability: {pstar[0]:.2%}')

Risk-neutral probability: 53.93%


### Solving the European Tree

In [5]:
treeV = treeAsset(funPayoff,tree,treeinfo,Z=Z,pstar=pstar)
treeV.style.format('{:.2f}')

,0,1,2
0,6.99,2.19,0.00
1,nan,13.38,5.00
2,nan,nan,24.67


### Compare to Black-Scholes

In [6]:
f0BS = bs_price(under=So,strike=K,T=T,rf=r,vol=sigma,option='put')
f0BS

6.8830357011256

## American Option

In [7]:
STYLE = 'american'
treeVamer, treeExer = treeAsset(funPayoff,tree,treeinfo,Z=Z,pstar=pstar,style='american')
treeVamer.style.format('{:.2f}')

,0,1,2
0,8.16,2.19,0.00
1,nan,16.06,5.00
2,nan,nan,24.67


In [8]:
treeExer

,0,1,2
0,NaN,NaN,NaN
1,NaN,True,NaN
2,NaN,NaN,NaN


### Compare all three

In [9]:
pd.DataFrame([f0BS,treeV.iloc[0,0],treeVamer.iloc[0,0]],columns=['value'],index=['BS value','tree value','American']).style.format('{:.4f}')

,value
BS value,6.8830
tree value,6.9865
American,8.1620


# Try a Two-Period Tree

Consider pricing a **put** option in a **2 period** tree. (Two periods of uncertainty, so with the starting point, three periods total.)
* $T=0.50$.
* $S_0=100$
* $K=110$
* $\sigma=.30$
* $r=0.02$

1. What are the stock prices at each node?
2. What is the risk-neutral probability?

### European style
1. What are the option prices at the final nodes?
2. Solve the tree for the option price at each node.

### American style
1. Suppose the option is American. Which node would you exercise early?
2. What is the value of the American option?

### Redo for a Call
How does the answer change?

Is there any path of early exercise?

### More nodes
Try solving with 30 time steps, still for $T=0.25$.